In [1]:
import requests
import numpy as np
import pandas as pd
import herepy
import config
import json
import isodate
from datetime import datetime as dt

In [5]:
residents = pd.read_csv("addresses_simulated_incomes_pitt.csv")
pittsburgh_mask = residents['MUNICIPALITY']=='PITTSBURGH'
pittsburgh = residents[pittsburgh_mask]
car_mask = pittsburgh['simulated_commute_car']==1
residents_car = pittsburgh[car_mask]
residents_transit = pittsburgh[-car_mask]
#Need to add store and vacant lot locations into a single df. Idk where current store info is--can you add that and the
#vacant lot data here?
lots = pd.read_csv("vacant_lots_centroid.csv")
stores = pd.read_csv("supermarkets.csv")

,OBJECTID,Company_Name,Address,City,State,ZIP_Code,NAICS_1,NAICS_1_Description,Latitude,Longitude,price score
0,1,5th Avenue Food Market,1701 5th Ave,Pittsburgh,PA,15219,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.438553,-79.982357,2
1,2,Aden Market,1318 Federal St # 1,Pittsburgh,PA,15212,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.457247,-80.006974,2
2,3,ALDI,2628 E Carson St,Pittsburgh,PA,15203,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.426989,-79.967410,1
3,4,ALDI,5200 Penn Ave,Pittsburgh,PA,15224,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.464217,-79.939334,1
4,5,ALDI,5631 Baum Blvd,Pittsburgh,PA,15206,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.467565,-79.922069,1
...,...,...,...,...,...,...,...,...,...,...,...
56,72,W N Grocery's,1018 W North Ave,Pittsburgh,PA,15233,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.453283,-80.020292,1
57,73,Whole Foods Market,5880 Centre Ave,Pittsburgh,PA,15206,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.458567,-79.928698,3
58,74,Wild Purveyors LLC,5308 Butler St # A,Pittsburgh,PA,15201,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.482271,-79.952199,3
59,75,Yi Chung Kwan Grocery,4701 5th Ave,Pittsburgh,PA,15213,445110,Supermarkets/Other Grocery (Exc Convenience) Strs,40.447235,-79.947104,2


In [13]:
#limited rows for testing. Change to residents_car.loc[:, [..]].values instead of the slices to use whole datasets
start_car_coords = residents_car.loc[:,["LATITUDE", "LONGITUDE"]].values
start_transit_coords = residents_transit.loc[:,["LATITUDE", "LONGITUDE"]].values
end_coords_lots = lots.loc[:,["Y", "X"]].values
end_coords_stores = stores.loc[:,["Latitude","Longitude"]].values
end_coords = np.concatenate((end_coords_lots,end_coords_stores),axis=0)

routingApi = herepy.RoutingApi(config.api_key)
publicTransitApi = herepy.PublicTransitApi(config.api_key)

In [14]:
# create blank lists that will be populated with trip characteristics
car_origin= []
car_destination= []
car_trip_miles= []
car_trip_time= []

#use car_api_start and car_api_end to time how long the API takes to run
car_api_start = dt.now()

#api is breaking without indicating the problematic row, so need to for loop it and wrap in try except
for i in range(0,len(start_car_coords)):
    for j in range(0,len(end_coords)):
        car_origin.append(i)
        car_destination.append(j)
        try:
            response_car = routingApi.matrix(
                    start_waypoints=start_car_coords[i:(i+1)],
                    destination_waypoints=end_coords[j:(j+1)],
                    departure='2020-08-04T17:00:00+02',
                    summary_attributes = ['traveltime','distance'],
                    modes=[herepy.RouteMode.fastest, herepy.RouteMode.car])
            #extract trip from response
            trip = response_car.as_dict()['response']['matrixEntry'][0]
            # store values
            car_trip_miles.append(trip['summary']['distance']/1609.34)
            car_trip_time.append(trip['summary']['travelTime']/3600)
        except:
            car_trip_miles.append(9999999)
            car_trip_time.append(999999999)


car_api_end = dt.now()


In [15]:

car_results = pd.DataFrame({"origin":car_origin,
              "destination":car_destination,
              "trip_distance":car_trip_miles,
              "trip_time":car_trip_time})
car_results['travel_cost'] = (car_results['trip_distance']*.575) + car_results['trip_time']*7.5

#store costs in i by k matrix
car_time_mat = car_results.pivot(index='origin',columns='destination',values='trip_time')
car_cost_mat = car_results.pivot(index='origin',columns='destination',values='travel_cost')

car_time_mat

destination,0,1,2,3,4,5,6,7,8,9,...,303,304,305,306,307,308,309,310,311,312
origin,,,,,,,,,,,,,,,,,,,,,
0,0.546389,0.297500,0.246667,0.434167,0.443611,0.330833,0.577222,0.333056,0.237500,0.350000,...,0.350000,0.397222,0.212778,0.195000,0.330000,0.269722,0.191944,0.016944,0.233889,0.328056
1,0.392500,0.269444,0.256667,0.352778,0.289722,0.302222,0.423333,0.251667,0.208889,0.292500,...,0.319444,0.319167,0.169722,0.199722,0.329722,0.279722,0.167778,0.214167,0.103333,0.244722
2,0.347778,0.376111,0.402500,0.056389,0.140833,0.408889,0.378611,0.123889,0.315556,0.397500,...,0.196389,0.154444,0.436389,0.466389,0.359722,0.321111,0.434444,0.424167,0.391667,0.352500
3,0.254167,0.355000,0.381389,0.154722,0.034722,0.387778,0.285000,0.119722,0.294444,0.290278,...,0.280000,0.051944,0.371111,0.402222,0.338611,0.300000,0.396944,0.403056,0.290278,0.245278
4,0.483611,0.249444,0.268056,0.299167,0.308611,0.282222,0.514444,0.198056,0.188889,0.289722,...,0.198333,0.262222,0.363333,0.386944,0.083611,0.144444,0.373056,0.300833,0.283889,0.244722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
430,0.549167,0.367500,0.342500,0.488889,0.452778,0.421389,0.581111,0.387778,0.328056,0.261667,...,0.440556,0.451944,0.127222,0.096944,0.420556,0.360278,0.094722,0.147778,0.190833,0.243056
431,0.452222,0.307778,0.334167,0.188056,0.266944,0.340556,0.483056,0.139444,0.247222,0.327500,...,0.098056,0.214167,0.368889,0.398889,0.288611,0.236389,0.366944,0.356667,0.321667,0.282500
432,0.415833,0.324722,0.351111,0.130000,0.208889,0.357500,0.446667,0.109722,0.264167,0.346111,...,0.136944,0.156111,0.385000,0.415000,0.316944,0.278333,0.383056,0.372778,0.340278,0.301111


In [16]:
#it took 8 and a half minutes to run 50 starts by 100 destinations, doing the starts in batches of 15
#should speed things up
(car_api_end-car_api_start).seconds/60

242.58333333333334

In [27]:
#try to use the public transit api for transit starting points, and if it fails
#because there is no public transit stop nearby, use walking directions
#create a blank list to store transit_times
transit_origins = []
transit_destinations = []
transit_times = []
transit_api_start = dt.now()
transit_fares = []
for i in range(0,len(start_transit_coords)):
    if i%20==0:
        print(i)
    for j in range(0,len(end_coords)):
        transit_origins.append(i)
        transit_destinations.append(j)
        try:
            response_transit = publicTransitApi.calculate_route(
                departure = list(start_transit_coords[i]),
                arrival = list(end_coords[j]),
                time = '2020-08-04T17:00:00+02')
            #extract trip_time from response
            trip_time = response_transit.as_dict()['Res']['Connections']['Connection'][0]['duration']
            # store trip time in hours
            transit_times.append(isodate.parse_duration(trip_time).seconds/3600)
            #add fare of 3.5
            transit_fares.append(3.5)
        except:
            response_ped = routingApi.matrix(
                start_waypoints=list(start_transit_coords[i:(i+1)]),
                destination_waypoints=list(end_coords[j:(j+1)]),
                departure='2020-08-04T17:00:00+02',
                summary_attributes = ['traveltime','distance'],
                modes=[herepy.RouteMode.fastest, herepy.RouteMode.pedestrian])
            
            trip = response_ped.as_dict()['response']['matrixEntry'][0]
            # if the call fails, it'll return a "status" in the response, to test for that
            if( 'status' in trip.keys()):
                #impute a very large trip time so 
                trip_time = 99999999999
            else: 
                trip_time = trip['summary']['travelTime']
            
            transit_times.append(trip_time/3600)
            #fare is 0 since they're walking
            transit_fares.append(0)
#timestamp
transit_api_end = dt.now()
#store results in tall dataframe
transit_results = pd.DataFrame({"origin":transit_origins,
                                "destination":transit_destinations,
                                "trip_time":transit_times,
                                "transit_fares":transit_fares})
#calculate trip cost using formula in paper
transit_results['trip_costs'] = (transit_results['trip_time']*7.5)+transit_results['transit_fares']

#store times and costs as i x k matrix
transit_time_mat = transit_results.pivot(index='origin',columns='destination',values='trip_time')
transit_cost_mat = transit_results.pivot(index='origin',columns='destination',values='trip_costs')

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280


In [30]:
transit_api_end-transit_api_start

datetime.timedelta(seconds=25174, microseconds=160584)

In [28]:
#combine time matrix and cost matrix
time_mat = pd.concat([car_time_mat,transit_time_mat])
#reset the row numbers
time_mat.reset_index(inplace=True)

cost_mat = pd.concat([car_cost_mat,transit_cost_mat])
cost_mat.reset_index(inplace=True)

#also combine the two household location matrices, which should now be the same number of rows
pittsburgh=pd.concat([residents_car,residents_transit])
pittsburgh.reset_index(inplace=True)


In [31]:
#save csvs of each
time_mat.to_csv('data/travel_times_all.csv')
cost_mat.to_csv('data/travel_costs_all.csv')
pittsburgh.to_csv('data/pittsburgh_all.csv')

Index(['origin',        0,        1,        2,        3,        4,        5,
              6,        7,        8,
       ...
            303,      304,      305,      306,      307,      308,      309,
            310,      311,      312],
      dtype='object', name='destination', length=314)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

0.7442

25

array([[ 40.4550461, -80.0528284],
       [ 40.4695965, -79.9169844]])

[40.455046100000004, -80.05282840000001]

array([[ 40.4550461, -80.0528284],
       [ 40.4695965, -79.9169844]])

destination,0,1,2,3,4
origin,,,,,
0,0.283889,0.330833,0.357222,0.141389,0.068611
1,0.611667,0.340278,0.298611,0.486111,0.495556
2,0.823611,0.552222,0.510556,0.698056,0.714444
3,0.831111,0.559722,0.518056,0.705556,0.721944
4,0.852500,0.581111,0.539444,0.726944,0.743333


In [70]:
residents_transit

,GEOID,FULL_ADDRESS,MUNICIPALITY,COUNTY,STATE,ZIP_CODE,LATITUDE,LONGITUDE,simulated_income,simulated_commute_car
4,42003562500,2913 ZEPHYR AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15204.0,40.455046,-80.052828,7533,0
5,42003110600,822 COLLINS AVE,PITTSBURGH,ALLEGHENY COUNTY,PA,15206.0,40.469597,-79.916984,6807,0
7,42003040500,258 1/2 MCKEE PL,PITTSBURGH,ALLEGHENY COUNTY,PA,15213.0,40.438568,-79.958795,55549,0
8,42003480200,1636 ELDERSLEE RD,BALDWIN BOROUGH,ALLEGHENY COUNTY,PA,15227.0,40.360828,-79.959813,94206,0
21,42003563000,1105 VILLAGE RD,PITTSBURGH,ALLEGHENY COUNTY,PA,15205.0,40.449962,-80.075243,49076,0
22,42003110600,6316 JACKSON ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15206.0,40.470284,-79.912368,69871,0
25,42003479000,571 DEWALT DR,BALDWIN TOWNSHIP,ALLEGHENY COUNTY,PA,15234.0,40.378669,-80.015922,951,0
26,42003488600,321 GLENCOE DR,WEST MIFFLIN BOROUGH,ALLEGHENY COUNTY,PA,15122.0,40.349946,-79.913236,3994,0
28,42003562400,226 CHALFONT ST,PITTSBURGH,ALLEGHENY COUNTY,PA,15210.0,40.416420,-80.003113,15152,0
29,42003492800,540 HALCOMB AVE,CLAIRTON,ALLEGHENY COUNTY,PA,15025.0,40.292152,-79.886965,4822,0


In [74]:
start_transit_coords

array([[ 40.4550461, -80.0528284],
       [ 40.4695965, -79.9169844]])

In [225]:
test = isodate.parse_duration(response_transit.as_dict()['Res']['Connections']['Connection'][0]['duration'])
test.seconds

5940

In [129]:
len(start_transit_coords)

166

In [154]:
trip_list[0]

{'startIndex': 0,
 'destinationIndex': 0,
 'summary': {'distance': 8918, 'travelTime': 1022, 'costFactor': 1042}}